In [35]:
import pandas as pd
import numpy as np
import datetime
import sklearn

## Lectura de datos

### Datos originales

In [62]:
# Postulantes
postulantes_educacion = pd.read_csv('./data/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv', low_memory=False)
postulantes_sexo_y_edad = pd.read_csv('./data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv', low_memory=False)
# Avisos
avisos_online = pd.read_csv('./data/datos_navent_fiuba/fiuba_5_avisos_online.csv', low_memory=False)
avisos_detalle = pd.read_csv('./data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv', low_memory=False)
# Transacciones
vistas = pd.read_csv('./data/datos_navent_fiuba/fiuba_3_vistas.csv', low_memory=False)
postulaciones = pd.read_csv('./data/datos_navent_fiuba/fiuba_4_postulaciones.csv', low_memory=False)

### Datos hasta 15 de abril

In [63]:
# Postulantes
h15_postulantes_educacion = pd.read_csv('./data/fiuba_hasta_15_abril/entrega6/fiuba_1_postulantes_educacion.csv', low_memory=False)
h15_postulantes_sexo_y_edad = pd.read_csv('./data/fiuba_hasta_15_abril/entrega6/fiuba_2_postulantes_genero_y_edad.csv', low_memory=False)
# Avisos
h15_avisos_online = pd.read_csv('./data/fiuba_hasta_15_abril/entrega6/fiuba_5_avisos_online.csv', low_memory=False)
h15_avisos_detalle = pd.read_csv('./data/fiuba_hasta_15_abril/entrega6/fiuba_6_avisos_detalle.csv', low_memory=False)
# Transacciones
h15_vistas = pd.read_csv('./data/fiuba_hasta_15_abril/entrega6/fiuba_3_vistas.csv', low_memory=False)
h15_postulaciones = pd.read_csv('./data/fiuba_hasta_15_abril/entrega6/fiuba_4_postulaciones.csv', low_memory=False)

### Datos desde el 15 de abril

In [64]:
# Postulantes
d15_postulantes_educacion = pd.read_csv('./data/fiuba_desde_15_Abril/fiuba_1_postulantes_educacion.csv', low_memory=False)
d15_postulantes_sexo_y_edad = pd.read_csv('./data/fiuba_desde_15_Abril/fiuba_2_postulantes_genero_y_edad.csv', low_memory=False)
# Avisos
d15_avisos_detalle = pd.read_csv('./data/fiuba_desde_15_Abril/fiuba_6_avisos_detalle.csv', low_memory=False)
# Transacciones
d15_vistas = pd.read_csv('./data/fiuba_desde_15_Abril/fiuba_3_vistas.csv', low_memory=False)

### Detalle de avisos faltantes

In [65]:
# Avisos
m_avisos_detalle = pd.read_csv('./data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv', low_memory=False)

## Merge de todos los datos

### Merge de los postulantes

Agregamos columna con ponderación del nivel de estudios alcanzado.

In [6]:
def categorizar_estudios(x):
    if((x is np.nan) | (x == 'Otro')):
        return 0
    if(x == 'Secundario'):
        return 1
    if(x == 'Terciario/Técnico'):
        return 2
    if(x == 'Universitario'):
        return 3
    if((x == 'Posgrado') | (x == 'Master') | (x == 'Doctorado')):
        return 4

postulantes_educacion['nivel_alcanzado'] = postulantes_educacion['nombre']\
    .apply(lambda x: categorizar_estudios(x))
h15_postulantes_educacion['nivel_alcanzado'] = h15_postulantes_educacion['nombre']\
    .apply(lambda x: categorizar_estudios(x))
d15_postulantes_educacion['nivel_alcanzado'] = d15_postulantes_educacion['nombre']\
    .apply(lambda x: categorizar_estudios(x))
#postulantes_educacion.head()

Unimos todos los dataframes de la educación de los postulantes y nos quedamos solo con los registros del máximo nivel de estudios alcanzado.

In [7]:
m_postulantes_educacion = pd.concat([postulantes_educacion, \
                                 h15_postulantes_educacion, \
                                 d15_postulantes_educacion])

In [8]:
m_postulantes_educacion = m_postulantes_educacion \
    .sort_values('nivel_alcanzado', ascending = False) \
    .drop_duplicates('idpostulante')

In [19]:
del m_postulantes_educacion['nombre']
m_postulantes_educacion.head(2)

,idpostulante,estado,nivel_alcanzado
0,NdJl,En Curso,4
237833,e4oaA2,En Curso,4


Unimos (concat) todos los dataframes de sexo y edad de los postulantes y luego los unimos (outer join) con los dataframes de los niveles de educación de los postulantes previamente tratados.

In [27]:
m_postulantes_sexo_y_edad = pd.concat([postulantes_sexo_y_edad, \
                                      h15_postulantes_sexo_y_edad, \
                                      d15_postulantes_sexo_y_edad])

In [28]:
m_postulantes_sexo_y_edad.count()

idpostulante       780020
fechanacimiento    745747
sexo               780020
dtype: int64

In [29]:
m_postulantes_sexo_y_edad = m_postulantes_sexo_y_edad.drop_duplicates()
m_postulantes_sexo_y_edad.count()

idpostulante       505382
fechanacimiento    478699
sexo               505382
dtype: int64

In [12]:
#m_postulantes_sexo_y_edad = m_postulantes_sexo_y_edad.groupby(['idpostulante'])\
#    .apply(lambda x: x.loc[x.sexo != 'NO_DECLARA',:] if len(x)>1 else x)\
#    .reset_index(drop=True)

In [30]:
m_postulantes_sexo_y_edad.count()

idpostulante       505382
fechanacimiento    478699
sexo               505382
dtype: int64

In [31]:
m_postulantes_sexo_y_edad = m_postulantes_sexo_y_edad.drop_duplicates('idpostulante')
m_postulantes_sexo_y_edad.count()

idpostulante       504407
fechanacimiento    477724
sexo               504407
dtype: int64

In [33]:
m_postulantes_sexo_y_edad['fechanacimiento'] = \
    pd.to_datetime(postulantes_sexo_y_edad['fechanacimiento'], errors='coerce')
m_postulantes_sexo_y_edad.count()

idpostulante       504407
fechanacimiento    384151
sexo               504407
dtype: int64

In [37]:
def calcular_edad(fecha_nacimiento):
    hoy = datetime.date.today()
    return hoy.year - fecha_nacimiento.year - ((hoy.month, hoy.day) < (fecha_nacimiento.month, fecha_nacimiento.day))

m_postulantes_sexo_y_edad['edad'] = m_postulantes_sexo_y_edad['fechanacimiento'].apply(lambda x: calcular_edad(x))
m_postulantes_sexo_y_edad.head()

,idpostulante,fechanacimiento,sexo,edad
0,NM5M,1970-12-03,FEM,47.0
1,5awk,1962-12-04,FEM,55.0
2,ZaO5,1978-08-10,FEM,39.0
3,NdJl,1969-05-09,MASC,49.0
4,eo2p,1981-02-16,MASC,37.0


In [41]:
edad_promedio = m_postulantes_sexo_y_edad['edad'].mean()
m_postulantes_sexo_y_edad['edad'].fillna(edad_promedio, inplace = True)
m_postulantes_sexo_y_edad.count()

idpostulante       504407
fechanacimiento    384151
sexo               504407
edad               504407
dtype: int64

In [42]:
del m_postulantes_sexo_y_edad['fechanacimiento']

In [43]:
m_postulantes_sexo_y_edad.head(2)

,idpostulante,sexo,edad
0,NM5M,FEM,47.0
1,5awk,FEM,55.0


In [44]:
m_postulantes = pd.merge(m_postulantes_sexo_y_edad, m_postulantes_educacion, \
                       on = 'idpostulante', how='outer')
m_postulantes.count()

idpostulante       504407
sexo               504407
edad               504407
estado             447909
nivel_alcanzado    447909
dtype: int64

In [47]:
nivel_promedio = int(m_postulantes.nivel_alcanzado.mean())
m_postulantes['nivel_alcanzado'].fillna(nivel_promedio, inplace = True)
m_postulantes.count()

idpostulante       504407
sexo               504407
edad               504407
estado             447909
nivel_alcanzado    504407
dtype: int64

In [48]:
m_postulantes.estado.value_counts()

Graduado      253833
En Curso      148072
Abandonado     46004
Name: estado, dtype: int64

In [49]:
def categorizar_estados(estado):
    if (estado == 'Graduado'):
        return 2
    if (estado == 'En Curso'):
        return 1
    if (estado == 'Abandonado'):
        return 0
m_postulantes['estado'] = m_postulantes['estado'].apply(lambda x : categorizar_estados(x))
m_postulantes.count()

idpostulante       504407
sexo               504407
edad               504407
estado             447909
nivel_alcanzado    504407
dtype: int64

In [51]:
estado_promedio = m_postulantes.estado.mean()
m_postulantes['estado'].fillna(estado_promedio, inplace = True)
m_postulantes.count()

idpostulante       504407
sexo               504407
edad               504407
estado             504407
nivel_alcanzado    504407
dtype: int64

In [53]:
m_postulantes.head(2)

,idpostulante,sexo,edad,estado,nivel_alcanzado
0,NM5M,FEM,47.0,2.0,1.0
1,5awk,FEM,55.0,2.0,3.0


### Merge de los avisos

In [66]:
avisos_detalle = avisos_detalle.loc[:,['idaviso', 'titulo', \
    'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',\
    'denominacion_empresa']]
h15_avisos_detalle = h15_avisos_detalle.loc[:,['idaviso', 'titulo', \
    'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',\
    'denominacion_empresa']]
d15_avisos_detalle = d15_avisos_detalle.loc[:,['idaviso', 'titulo', \
    'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',\
    'denominacion_empresa']]
m_avisos_detalle = m_avisos_detalle.loc[:,['idaviso', 'titulo', \
    'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',\
    'denominacion_empresa']]

In [67]:
m_avisos = pd.concat([avisos_detalle, h15_avisos_detalle, \
                      d15_avisos_detalle, m_avisos_detalle])

In [68]:
m_avisos.count()

idaviso                 45969
titulo                  45969
nombre_zona             45969
tipo_de_trabajo         45969
nivel_laboral           45634
nombre_area             45969
denominacion_empresa    45955
dtype: int64

In [69]:
m_avisos = m_avisos.drop_duplicates()
m_avisos.count()

idaviso                 25637
titulo                  25637
nombre_zona             25637
tipo_de_trabajo         25637
nivel_laboral           25302
nombre_area             25637
denominacion_empresa    25630
dtype: int64

In [70]:
m_avisos = m_avisos.drop_duplicates('idaviso')
m_avisos.count()

idaviso                 25288
titulo                  25288
nombre_zona             25288
tipo_de_trabajo         25288
nivel_laboral           24953
nombre_area             25288
denominacion_empresa    25281
dtype: int64

In [71]:
m_avisos.head(2)

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,Enfermeras,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste


In [72]:
m_avisos.nivel_laboral.value_counts()

Senior / Semi-Senior                    16983
Junior                                   4149
Otro                                     1972
Jefe / Supervisor / Responsable          1527
Gerencia / Alta Gerencia / Dirección      322
Name: nivel_laboral, dtype: int64

In [73]:
# Completo los NaNs del nivel laboral con "Senior / Semi-Senior"
m_avisos['nivel_laboral'].fillna('Senior / Semi-Senior', inplace = True)
m_avisos.count()

idaviso                 25288
titulo                  25288
nombre_zona             25288
tipo_de_trabajo         25288
nivel_laboral           25288
nombre_area             25288
denominacion_empresa    25281
dtype: int64

In [76]:
# Completo los NaNs del nombre de la empresa con "No declara"
m_avisos['denominacion_empresa'].fillna('No declara', inplace = True)
m_avisos.count()

idaviso                 25288
titulo                  25288
nombre_zona             25288
tipo_de_trabajo         25288
nivel_laboral           25288
nombre_area             25288
denominacion_empresa    25288
dtype: int64

In [77]:
m_avisos.head(2)

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,Enfermeras,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste


In [79]:
del m_avisos['titulo']

### Label encoding

In [78]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [80]:
avisos_encoding = MultiColumnLabelEncoder(columns = \
    ['nombre_zona','tipo_de_trabajo', 'nivel_laboral', \
     'nombre_area', 'denominacion_empresa']).fit_transform(m_avisos)
avisos_encoding.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,7,1,4,30,4005
1,17903700,7,1,4,158,1640
2,1000150677,1,1,4,181,1561
3,1000610287,7,1,4,181,4119
4,1000872556,7,1,4,143,1267


In [81]:
postulantes_encoding = MultiColumnLabelEncoder(columns = \
    ['sexo']).fit_transform(m_postulantes)
postulantes_encoding.head()

,idpostulante,sexo,edad,estado,nivel_alcanzado
0,NM5M,1,47.0,2.0,1.0
1,5awk,1,55.0,2.0,3.0
2,ZaO5,1,39.0,0.0,3.0
3,NdJl,2,49.0,1.0,4.0
4,eo2p,2,37.0,2.0,4.0


### Merge de las postulaciones

In [82]:
m_postulaciones = pd.concat([postulaciones, h15_postulaciones])

In [87]:
m_postulaciones['fechapostulacion'] = \
    pd.to_datetime(m_postulaciones['fechapostulacion'], errors='coerce')
m_postulaciones.count()

idaviso             8311264
idpostulante        8311264
fechapostulacion    8311264
dtype: int64

In [91]:
m_postulaciones['dia'] = m_postulaciones['fechapostulacion'].dt.weekday
m_postulaciones['hora'] = m_postulaciones['fechapostulacion'].dt.hour
del m_postulaciones['fechapostulacion']
m_postulaciones.head(2)

,idaviso,idpostulante,dia,hora
0,1112257047,NM5M,0,16
1,1111920714,NM5M,1,9


In [96]:
postulantes_encoding.count()

idpostulante       504407
sexo               504407
edad               504407
estado             504407
nivel_alcanzado    504407
dtype: int64

In [97]:
merged = pd.merge(m_postulaciones, postulantes_encoding, \
                  on = 'idpostulante', how = 'inner')

In [98]:
merged.count()

idaviso            8311264
idpostulante       8311264
dia                8311264
hora               8311264
sexo               8311264
edad               8311264
estado             8311264
nivel_alcanzado    8311264
dtype: int64

In [99]:
avisos_encoding.count()

idaviso                 25288
nombre_zona             25288
tipo_de_trabajo         25288
nivel_laboral           25288
nombre_area             25288
denominacion_empresa    25288
dtype: int64

In [100]:
merged2 = pd.merge(merged, avisos_encoding, on = 'idaviso', how = 'inner')
merged2.count()

idaviso                 7742942
idpostulante            7742942
dia                     7742942
hora                    7742942
sexo                    7742942
edad                    7742942
estado                  7742942
nivel_alcanzado         7742942
nombre_zona             7742942
tipo_de_trabajo         7742942
nivel_laboral           7742942
nombre_area             7742942
denominacion_empresa    7742942
dtype: int64

In [101]:
merged2.head()

,idaviso,idpostulante,dia,hora,sexo,edad,estado,nivel_alcanzado,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1112257047,NM5M,0,16,1,47.0,2.0,1.0,7,2,3,14,1369
1,1112257047,1kJqGb,0,10,1,34.0,1.0,3.0,7,2,3,14,1369
2,1112257047,eOE9Rr,0,10,1,39.0,0.0,3.0,7,2,3,14,1369
3,1112257047,Zrx8Xz,1,10,1,39.0,2.0,2.0,7,2,3,14,1369
4,1112257047,ZrKNQY,1,17,1,32.0,1.0,3.0,7,2,3,14,1369


## KNN

In [104]:
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

c:\users\i860341\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [102]:
X = merged2[['nivel_alcanzado', 'sexo','estado','dia','hora','edad',\
    'nombre_zona','tipo_de_trabajo','nivel_laboral','nombre_area','denominacion_empresa']]
Y = merged2['idaviso']

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=7506)

In [106]:
print("Tamaño de X:\t\t", len(X), "\nTamaño de X_train:\t", len(X_train), "\nTamaño de X_test:\t", len(X_test), \
      "\nTamaño de y:\t\t", len(Y), "\nTamaño de y_train:\t", len(y_train), "\nTamaño de y_test:\t", len(y_test))

Tamaño de X:		 7742942 
Tamaño de X_train:	 6968647 
Tamaño de X_test:	 774295 
Tamaño de y:		 7742942 
Tamaño de y_train:	 6968647 
Tamaño de y_test:	 774295


In [ ]:
# Create KNN classifier
knn = KNeighborsClassifier(n_neighbors=7)

# FIT (this does nothing in normal KNN)
knn.fit(X_train, y_train)

# predict the response
pred = knn.predict(X_test)
pred

In [210]:
# Create KNN prophet
knn_p = KNeighborsClassifier(n_neighbors=7)

# FIT (this does nothing in normal KNN)
knn_p.fit(X_train, y_train)

# predict the response
prophecy = knn_p.predict_proba(X_test)
prophecy[1-10]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.14285714, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.42857143, 0.        , 0.        ,
       0.        , 0.14285714, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.14285714, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.14285714,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [226]:
y_test.to_frame().groupby('idaviso').size()

idaviso
12812680       18
1000132160      2
1000139175     56
1000365356     31
1000365835     21
1000371968      2
1110007485     32
1111901937     39
1112020586      2
1112020797     62
1112035895    121
1112101220     71
1112117283      3
1112121823     20
1112157766      5
1112222925     37
1112233938      9
1112235972     27
1112241594    252
1112260031      6
1112264647    355
1112266470     48
1112267991     48
1112268555     98
1112269727     59
1112277365     33
1112277486     26
1112278821     16
1112281170     14
1112291646     16
             ... 
1112309768     28
1112320808     23
1112322878    100
1112324086      5
1112350652     17
1112366270     20
1112370898      2
1112371102      4
1112373820     68
1112374826     20
1112378180     22
1112389115     18
1112397938     23
1112404197      6
1112407104      3
1112407469      5
1112407528      4
1112408665      3
1112417400      9
1112420914     15
1112424300     32
1112428486     28
1112432407     85
1112432658      8
11

In [232]:
len(prophecy[0])

66

In [67]:
test_final = pd.read_csv('./data/test_final_100k.csv', low_memory=False)
test_final.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [65]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
id              100000 non-null int64
idaviso         100000 non-null int64
idpostulante    100000 non-null object
dtypes: int64(2), object(1)
memory usage: 2.3+ MB
